In [1]:
import keras

Using TensorFlow backend.
/home/jared/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Data Prep

In [2]:
%cd data

data_dir = %pwd
train_dir = data_dir + '/train/'
valid_dir = data_dir + '/valid/'
test_dir = data_dir + '/test/'
sample_dir = data_dir + '/sample/'

/home/jared/courses/deeplearning2/Kaggle/dogscats/data


In [3]:
import os
from glob import glob
import numpy as np
from shutil import copyfile

num_valid = 1000
num_sample = 64

classes = ['cat', 'dog']


In [28]:
for cls in classes:
    os.mkdir(train_dir + cls, 775)
    os.mkdir(valid_dir + cls, 775)
    os.mkdir(sample_dir + 'train/' + cls, 775)
    os.mkdir(sample_dir + 'valid/' + cls, 775)    

In [29]:

for cls in classes:
    files = glob(train_dir + cls + '*.jpg')
    np.random.shuffle(files)

    for file in files[:num_sample]:
        copyfile(file, sample_dir + 'train/' + cls + '/' + os.path.basename(file))
        
    for file in files[num_sample:num_sample+num_sample]:
        copyfile(file, sample_dir + 'valid/' + cls + '/' + os.path.basename(file))
    
    for file in files[:num_valid]:
        os.rename(file, valid_dir + cls + '/' + os.path.basename(file))
        
    for file in files[num_valid:]:
        os.rename(file, train_dir + cls + '/' + os.path.basename(file))
    


# Simple Model

In [4]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, Convolution2D, MaxPooling2D, Input
from keras.optimizers import Adam

#base_dir = sample_dir
path = data_dir + '/'
batch_size = 64
image_size = (224,224)
image_shape = (*image_size, 3)

In [5]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator()
val_gen = ImageDataGenerator()

batches = gen.flow_from_directory(path+'train/', target_size=image_size, batch_size=batch_size, shuffle=True)
val_batches = val_gen.flow_from_directory(path+'valid/', target_size=image_size, batch_size=batch_size, shuffle=True)

train_steps = batches.n//batch_size
val_steps = val_batches.n//batch_size

if (val_steps == 0):
    val_steps = 1

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [104]:
model = Sequential([
    BatchNormalization(input_shape=image_shape)
    , Convolution2D(64, (3,3), activation='relu')
    , MaxPooling2D((2,2))
    , Convolution2D(64, (3,3), activation='relu')
    , MaxPooling2D((2,2))
    , Flatten()
    , Dense(124, activation='relu')
    , Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 57600)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 124)               7142524   
__________

In [96]:
model.fit_generator(batches, train_steps, epochs=1, validation_data=val_batches, validation_steps=val_steps)

Epoch 1/1
359/359 [==============================] - 75s - loss: 0.6704 - acc: 0.6494 - val_loss: 0.5757 - val_acc: 0.6699


# Transfer Learning

In [6]:
batches = gen.flow_from_directory(path+'train/', target_size=image_size, batch_size=batch_size, shuffle=False)
val_batches = val_gen.flow_from_directory(path+'valid/', target_size=image_size, batch_size=batch_size, shuffle=False)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
from keras.applications.vgg16 import VGG16

vgg = VGG16(include_top=False, input_shape=image_shape, pooling='avg')
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
X_train = vgg.predict_generator(batches, train_steps+1)
y_train = keras.utils.np_utils.to_categorical(batches.classes)

In [9]:
X_valid = vgg.predict_generator(val_batches, val_steps+1)
y_valid = val_batches.classes

Found 23000 images belonging to 2 classes.


In [14]:
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

In [13]:
save_array(path + 'train_vgg_average_pool_features.bc', X_train)
save_array(path + 'valid_vgg_average_pool_features.bc', X_valid)

print(X_train.shape, X_valid.shape)

(23000, 512) (2000, 512)


In [16]:
X_train = load_array(path + 'train_vgg_average_pool_features.bc')
X_valid = load_array(path + 'valid_vgg_average_pool_features.bc')

print(X_train.shape, X_valid.shape)

(23000, 512) (2000, 512)


In [17]:
y_train = keras.utils.np_utils.to_categorical(batches.classes)
y_valid = keras.utils.np_utils.to_categorical(val_batches.classes)

In [18]:
model = Sequential([
    BatchNormalization(input_shape=vgg.output_shape[1:])
    , Dense(512, activation='relu')
    , BatchNormalization()
    , Dropout(.5)
    , Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 267,778
Trainable params: 265,730
Non-trainable params: 2,048
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, epochs=4, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/4
23000/23000 [==============================] - 5s - loss: 0.1541 - acc: 0.9453 - val_loss: 0.0798 - val_acc: 0.9705
Epoch 2/4
23000/23000 [==============================] - 4s - loss: 0.1004 - acc: 0.9604 - val_loss: 0.0691 - val_acc: 0.9745
Epoch 3/4
23000/23000 [==============================] - 4s - loss: 0.0930 - acc: 0.9645 - val_loss: 0.0694 - val_acc: 0.9730
Epoch 4/4
23000/23000 [==============================] - 4s - loss: 0.0848 - acc: 0.9661 - val_loss: 0.0736 - val_acc: 0.9710


In [93]:
model.optimizer.lr = .0001
model.fit(X_train, y_train, epochs=8, validation_data=(X_valid, y_valid))

Train on 22976 samples, validate on 1984 samples
Epoch 1/8
22976/22976 [==============================] - 5s - loss: 0.1477 - acc: 0.9397 - val_loss: 0.1325 - val_acc: 0.9380
Epoch 2/8
22976/22976 [==============================] - 5s - loss: 0.1377 - acc: 0.9441 - val_loss: 0.1468 - val_acc: 0.9380
Epoch 3/8
22976/22976 [==============================] - 5s - loss: 0.1256 - acc: 0.9487 - val_loss: 0.1411 - val_acc: 0.9370
Epoch 4/8
22976/22976 [==============================] - 5s - loss: 0.1173 - acc: 0.9543 - val_loss: 0.1496 - val_acc: 0.9345
Epoch 5/8
22976/22976 [==============================] - 5s - loss: 0.1120 - acc: 0.9560 - val_loss: 0.1334 - val_acc: 0.9486
Epoch 6/8
22976/22976 [==============================] - 5s - loss: 0.1032 - acc: 0.9594 - val_loss: 0.1418 - val_acc: 0.9410
Epoch 7/8
22976/22976 [==============================] - 5s - loss: 0.1092 - acc: 0.9574 - val_loss: 0.1374 - val_acc: 0.9405
Epoch 8/8
22976/22976 [==============================] - 5s - loss: 0

In [88]:
model.optimizer.lr = .00001
model.fit(X_train, y_train, epochs=12, validation_data=(X_valid, y_valid))

Train on 22976 samples, validate on 1984 samples
Epoch 1/12
22976/22976 [==============================] - 3s - loss: 0.0970 - acc: 0.9625 - val_loss: 0.1490 - val_acc: 0.9420
Epoch 2/12
22976/22976 [==============================] - 3s - loss: 0.0951 - acc: 0.9646 - val_loss: 0.1767 - val_acc: 0.9405
Epoch 3/12
22976/22976 [==============================] - 3s - loss: 0.0961 - acc: 0.9640 - val_loss: 0.1651 - val_acc: 0.9476
Epoch 4/12
22976/22976 [==============================] - 3s - loss: 0.0913 - acc: 0.9659 - val_loss: 0.2074 - val_acc: 0.9400
Epoch 5/12
22976/22976 [==============================] - 3s - loss: 0.0804 - acc: 0.9713 - val_loss: 0.1729 - val_acc: 0.9430
Epoch 6/12
22976/22976 [==============================] - 3s - loss: 0.0784 - acc: 0.9711 - val_loss: 0.1854 - val_acc: 0.9446
Epoch 7/12
22976/22976 [==============================] - 3s - loss: 0.0752 - acc: 0.9725 - val_loss: 0.1689 - val_acc: 0.9496
Epoch 8/12
22976/22976 [==============================] - 3s -

In [89]:
model.optimizer.lr = .000001
model.fit(X_train, y_train, epochs=16, validation_data=(X_valid, y_valid))

Train on 22976 samples, validate on 1984 samples
Epoch 1/16
22976/22976 [==============================] - 3s - loss: 0.0652 - acc: 0.9779 - val_loss: 0.1988 - val_acc: 0.9415
Epoch 2/16
22976/22976 [==============================] - 3s - loss: 0.0668 - acc: 0.9772 - val_loss: 0.2153 - val_acc: 0.9476
Epoch 3/16
22976/22976 [==============================] - 3s - loss: 0.0605 - acc: 0.9785 - val_loss: 0.2109 - val_acc: 0.9456
Epoch 4/16
22976/22976 [==============================] - 3s - loss: 0.0574 - acc: 0.9807 - val_loss: 0.2363 - val_acc: 0.9425
Epoch 5/16
22976/22976 [==============================] - 3s - loss: 0.0589 - acc: 0.9798 - val_loss: 0.2386 - val_acc: 0.9456
Epoch 6/16
22976/22976 [==============================] - 3s - loss: 0.0612 - acc: 0.9794 - val_loss: 0.2152 - val_acc: 0.9466
Epoch 7/16
22976/22976 [==============================] - 3s - loss: 0.0562 - acc: 0.9815 - val_loss: 0.2216 - val_acc: 0.9491
Epoch 8/16
22976/22976 [==============================] - 3s -

# Data Augmentation

In [27]:
def get_augmented_images(path, image_size, batch_size):
    gen = ImageDataGenerator(
        rotation_range=90.,
        zoom_range=2.,
        fill_mode='nearest',
        horizontal_flip=True,
        vertical_flip=True)
    
    return gen.flow_from_directory(path, target_size=image_size, batch_size=batch_size, shuffle=False)

In [28]:
X_train_aug_1 = vgg.predict_generator(get_augmented_images(path+'train/', image_size, batch_size), train_steps+1)
X_train_aug_2 = vgg.predict_generator(get_augmented_images(path+'train/', image_size, batch_size), train_steps+1)
X_train_aug_3 = vgg.predict_generator(get_augmented_images(path+'train/', image_size, batch_size), train_steps+1)

Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


In [29]:
save_array(path + 'train_vgg_average_pool_features_augmented_1.bc', X_train_aug_1)
save_array(path + 'train_vgg_average_pool_features_augmented_2.bc', X_train_aug_2)
save_array(path + 'train_vgg_average_pool_features_augmented_3.bc', X_train_aug_3)

In [41]:
y_train = keras.utils.np_utils.to_categorical(batches.classes)

In [42]:
X_train = load_array(path + 'train_vgg_average_pool_features.bc')
X_train_aug_1 = load_array(path + 'train_vgg_average_pool_features_augmented_1.bc')
X_train_aug_2 = load_array(path + 'train_vgg_average_pool_features_augmented_2.bc')
X_train_aug_3 = load_array(path + 'train_vgg_average_pool_features_augmented_3.bc')

print(X_train.shape, X_train_aug_1.shape)

(23000, 512) (23000, 512)


In [43]:
X_valid = load_array(path + 'valid_vgg_average_pool_features.bc')
y_valid = keras.utils.np_utils.to_categorical(val_batches.classes)

In [52]:
model = Sequential([
    BatchNormalization(input_shape=vgg.output_shape[1:])
    , Dropout(.5)
    , Dense(512, activation='relu')
    , BatchNormalization()
    , Dropout(.5)
    , Dense(1024, activation='relu')
    , BatchNormalization()
    , Dropout(.5)
    , Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_9 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_10 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              525312    
_________________________________________________________________
batch_normalization_11 (Batc (None, 1024)              4096      
__________

In [53]:
model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 7s - loss: 0.2623 - acc: 0.9118 - val_loss: 0.0886 - val_acc: 0.9670
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.6468 - acc: 0.6551 - val_loss: 0.1873 - val_acc: 0.9390
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5970 - acc: 0.6783 - val_loss: 0.1258 - val_acc: 0.9535
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5834 - acc: 0.6864 - val_loss: 0.1276 - val_acc: 0.9460


In [54]:
model.optimizer.lr = .0001
for _ in range(2):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1656 - acc: 0.9324 - val_loss: 0.0833 - val_acc: 0.9695
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5979 - acc: 0.6834 - val_loss: 0.1280 - val_acc: 0.9515
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5824 - acc: 0.6868 - val_loss: 0.1737 - val_acc: 0.9415
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5700 - acc: 0.6970 - val_loss: 0.1324 - val_acc: 0.9490
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1537 - acc: 0.9357 - val_loss: 0.0785 - val_acc: 0.9685
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5791 -

In [ ]:
model.optimizer.lr = .00001
for _ in range(3):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1558 - acc: 0.9382 - val_loss: 0.0814 - val_acc: 0.9710
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5673 - acc: 0.6999 - val_loss: 0.1169 - val_acc: 0.9630
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5551 - acc: 0.7021 - val_loss: 0.1268 - val_acc: 0.9580
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5464 - acc: 0.7116 - val_loss: 0.1122 - val_acc: 0.9635
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1511 - acc: 0.9394 - val_loss: 0.0814 - val_acc: 0.9710
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5565 -

In [ ]:
model.optimizer.lr = .000001
for _ in range(5):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1445 - acc: 0.9420 - val_loss: 0.0736 - val_acc: 0.9700
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5479 - acc: 0.7107 - val_loss: 0.2146 - val_acc: 0.9075
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5341 - acc: 0.7199 - val_loss: 0.1111 - val_acc: 0.9585
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
19680/23000 [========================>.....] - ETA: 0s - loss: 0.5270 - acc: 0.7206